In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark Introduction").\
                            config("spark.some.config.option", "some-value").\
                            config("spark.master","local[1]").\
                            config("spark.sql.debug.maxToStringFields", 1000).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/12 10:26:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

data2 = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data2,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



In [3]:
import hashlib

def encrypt_name(name: str):
    if name is None:
        return 9999
    enc_value = hashlib.sha256(name.encode()).hexdigest()
    return enc_value

In [4]:
from pyspark.sql.functions import udf 
encrypt_udf = udf(encrypt_name)

In [13]:
enc_cust_df = df.withColumn('middlename', encrypt_udf('middlename'))

enc_cust_df = enc_cust_df.withColumn('lastname', encrypt_udf('lastname'))

In [14]:
enc_cust_df.show(5)

+---------+--------------------+--------------------+-----+------+------+
|firstname|          middlename|            lastname|   id|gender|salary|
+---------+--------------------+--------------------+-----+------+------+
|    James|e3b0c44298fc1c149...|9f542590100424c92...|36636|     M|  3000|
|  Michael|e10d850954a8e6113...|e3b0c44298fc1c149...|40288|     M|  4000|
|   Robert|e3b0c44298fc1c149...|6e8d8801f51aafe84...|42114|     M|  4000|
|    Maria|9a474836482002bd0...|fe16a94194b3857b5...|39192|     F|  4000|
|      Jen|aebac53c46bbeff10...|18f285592896ec8ea...|     |     F|    -1|
+---------+--------------------+--------------------+-----+------+------+



22/12/12 11:17:14 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 498121 ms exceeds timeout 120000 ms
22/12/12 11:17:14 WARN SparkContext: Killing executors is not supported by current scheduler.
